# Retrieval-Augmented Shopping Assistant - EDA

This notebook explores the ABO dataset for initial insights.

In [ ]:
import pandas as pd

In [ ]:
# Load ABO image dataset
df_img = pd.read_csv("/kaggle/input/amazon-berkeley-objects/images/metadata/images.csv")  # or csv if applicable

In [ ]:
# 1. Check structure
print(df_img.shape)
print(df_img.columns)
print(df_img.dtypes)

# 2. Check missing values
print(df_img.notnull().sum())

# 3. Sample record
df_img.sample(5)

# 4. (If possible) display an image
from PIL import Image
import matplotlib.pyplot as plt

img_loc = str(df_img.loc[df_img['path']== '8c/8ccb5859.jpg']['path']).split()[1]
img_path = '/kaggle/input/amazon-berkeley-objects/images/small/' + img_loc  # based on image_id field
img = Image.open(img_path)
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
# Load ABO metadata dataset

import os, glob
full_metadata = pd.DataFrame()
for json_file in glob.glob("/kaggle/input/listing/listings/metadata/*.json"):
    print('Loading file: ' + json_file + '\n')
    df_metadata = pd.read_json(json_file, lines = True)
    full_metadata = pd.concat([full_metadata, df_metadata])

In [ ]:
# 1. Check structure
print(full_metadata.shape)
print(full_metadata.columns)
print(full_metadata.dtypes)

In [ ]:
# 2. Check missing values
full_metadata.isna().sum().sort_values(ascending=False)

From above, "finish_type" column has the lowest number of values present i.e. 1536 (147702 - 146166)  out of 147702 product entries.

In [ ]:
#Find duplicate item_id rows

duplicate_counts = full_metadata['item_id'].value_counts()
duplicate_counts = duplicate_counts[duplicate_counts > 1]
print(duplicate_counts)

There are duplicates but analyising one of the item_id below.

In [ ]:
full_metadata.loc[full_metadata['item_id'] == 'B01BC2TBZ4']

There are duplicate item_ids, because of multiple language support. We will remove duplicates after creating 
a scoped dataframe for supported language.

In [ ]:
# 3. Sample record
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
#full_metadata.sample(5)

From above, it is clear that:
1. Only **item_name** column is present for all products.
2. We will use **item_name** to filter for in scope language - English.

In [ ]:
# 4. Display an image using image id from the metadata

from PIL import Image
import matplotlib.pyplot as plt
print(full_metadata.loc[full_metadata['main_image_id'] == '413jqtAkNSL'])

In [ ]:
img_row = full_metadata.loc[full_metadata['main_image_id'] == '413jqtAkNSL']
img_id = img_row.iloc[0]['main_image_id']
img_location = str(df_img.loc[df_img['image_id'] == img_id]['path'])
img = Image.open('/kaggle/input/amazon-berkeley-objects/images/small/' + img_location.split()[1])
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
# Analyse the number of languages supported. US-English in scope only.
count_US_lang_key = 0
count_prodDesc = 0
count_total=0
lang_set = set()
for val in full_metadata['item_name']:
    count_total += 1
    for key, value in val[0].items():
        if(key == 'language_tag'):
            lang_set.add(value)
print(lang_set)

Amongst these, only English language will be used. i.e,
en_SG,
en_CA,
en_AU,
en_GB, 
en_AE, 
en_US,
en_IN

In [ ]:
valid_languages = {'en_SG', 'en_CA', 'en_AU', 'en_GB', 'en_AE', 'en_US', 'en_IN'}

countTotalProdDesc = 0
countTotalEngDesc = 0

for val in full_metadata['item_name']:
    if isinstance(val, list):
        for item in val:
            if isinstance(item, dict):
                countTotalProdDesc += 1
                if item.get('language_tag') in valid_languages:
                    countTotalEngDesc += 1

print('Total language-tagged entries (all languages):', countTotalProdDesc)
print('Total in-scope English entries:', countTotalEngDesc)
print('Percentage of data under scope:', round(countTotalEngDesc / countTotalProdDesc * 100, 2), '%')

In [ ]:
# Reduce the metadata to in scope languages only
def has_valid_language(item_name):
    if isinstance(item_name, list):
        return any(
            isinstance(entry, dict) and entry.get('language_tag') in valid_languages
            for entry in item_name
        )
    return False

inScopeMetadata = full_metadata[full_metadata['item_name'].apply(has_valid_language)].reset_index(drop=True)

In [ ]:
print(inScopeMetadata.shape)

In [ ]:
from collections import Counter

lang_counter = Counter()

for val in full_metadata['item_name']:
    if isinstance(val, list):
        for item in val:
            if isinstance(item, dict):
                lang = item.get('language_tag')
                if lang:
                    lang_counter[lang] += 1

# Convert to sorted list
sorted_langs = sorted(lang_counter.items(), key=lambda x: x[1], reverse=True)

# Print
print("Language-wise distribution in `item_name` field:\n")
for lang, count in sorted_langs:
    print(f"{lang}: {count}")

In [ ]:
def get_first_valid_lang(item_name):
    if isinstance(item_name, list):
        for entry in item_name:
            if isinstance(entry, dict) and entry.get('language_tag') in valid_languages:
                return entry['language_tag']
    return None

full_metadata['language_matched'] = full_metadata['item_name'].apply(get_first_valid_lang)

In [ ]:
print(f"Total rows with valid English variants: {len(inScopeMetadata)} / {len(full_metadata)}")

In [ ]:
#Find duplicate item_id rows

duplicate_counts = inScopeMetadata['item_id'].value_counts()
duplicate_counts = duplicate_counts[duplicate_counts > 1]
print(duplicate_counts)

In [ ]:
inScopeMetadata.loc[inScopeMetadata['item_id'] == 'B07WC622LH']

In [ ]:
def auto_flatten_json_columns(df, keys_to_try=['value', 'name', 'node_name']):
    """
    Detects and flattens columns containing lists of JSON objects,
    extracting specified keys.
    Adds new columns with a `_flat` suffix.
    """
    def extract_from_list(ld, keys):
        if isinstance(ld, list):
            for key in keys:
                values = [str(d.get(key)) for d in ld if isinstance(d, dict) and key in d]
                if values:  # found at least one valid value
                    return ", ".join(values)
        return None

    # Track flattened columns
    flattened = []

    for col in df.columns:
        sample = df[col].iloc[0]
        if isinstance(sample, list) and all(isinstance(i, dict) for i in sample):
            flat_col = f"{col}_flat"
            df[flat_col] = df[col].apply(lambda x: extract_from_list(x, keys_to_try))
            flattened.append(flat_col)

    return df, flattened

In [ ]:
print(inScopeMetadata.shape)

In [ ]:
inScopeMetadata, flattened_cols = auto_flatten_json_columns(inScopeMetadata)
print("Flattened columns:", flattened_cols)


In [ ]:
print(inScopeMetadata.columns)

In [ ]:
print(full_metadata.shape)

In [ ]:
full_metadata, flattened_cols = auto_flatten_json_columns(full_metadata)
print("Flattened columns:", flattened_cols)

In [ ]:
print(full_metadata.shape)

In [ ]:
print(full_metadata.dtypes)

In [ ]:
# 2. Check missing values
print(full_metadata.notnull().sum())

In [ ]:
print(full_metadata.index[:10])

In [ ]:
full_metadata = full_metadata.reset_index(drop=True)

In [ ]:
print(full_metadata.loc[1])  # Row with index label 1

In [ ]:
print(full_metadata.columns)

In [ ]:
#1. Top 10 Brands

import matplotlib.pyplot as plt
import seaborn as sns

top_brands = full_metadata['brand_flat'].value_counts().head(10)

plt.figure(figsize=(10, 6))
sns.barplot(x=top_brands.values, y=top_brands.index, palette='viridis')
plt.title("Top 10 Brands by Frequency")
plt.xlabel("Count")
plt.ylabel("Brand")
plt.tight_layout()
plt.show()

In [ ]:
# Top 10 product types

top_types = full_metadata['product_type_flat'].value_counts().head(10)

plt.figure(figsize=(10, 6))
sns.barplot(x=top_types.values, y=top_types.index, palette='mako')
plt.title("Top 10 Product Types")
plt.xlabel("Count")
plt.ylabel("Product Type")
plt.tight_layout()
plt.show()

In [ ]:
#Word cloud from bullet_point information

from wordcloud import WordCloud

text = " ".join(full_metadata['bullet_point_flat'].dropna().astype(str).values)
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Common Words in Bullet Points")
plt.show()

In [ ]:
#Word cloud from node information

from wordcloud import WordCloud

text = " ".join(full_metadata['node_flat'].dropna().astype(str).values)
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Common Words in Node")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=full_metadata, x='domain_name', order=full_metadata['domain_name'].value_counts().index)
plt.title("Items per Domain")
plt.xlabel("Domain")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Above plots show:
1. More than half of the items are on "Amazon" marketplace.
2. Cellular phone Case, Shoes, Grocery are top 3 product types.
3. Amazon.in accounts for more than half of listings.
4. Words around mobile accessories dominate the product's catefories description column (node).
5. Multiple languages are supported per product. Languages list:
{'ko_KR', 'es_MX', 'zh_TW', 'sv_SE', 'en_AU', 'fr_CA', 'en_CA', 'he_IL', 'es_US', 'kn_IN', 'tr_TR', 'hi_IN', 'zh_CN', 'en_GB', 'nl_NL', 'pt_BR', 'fr_FR', 'te_IN', 'ja_JP', 'en_IN', 'es_ES', 'en_US', 'ar_AE', 'en_SG', 'ml_IN', 'en_AE', 'cs_CZ', 'it_IT', 'pl_PL', 'de_DE'}

The full_metadata will be used

In [ ]:
#Serialize to parquet/persistent storage

inScopeMetadata.to_parquet("inScopeMetadata_with_embeddings.parquet", index=False)


With above copy, we have:

inScopeMetadata_with_embeddings.parquet → all metadata + embedding inputs